In [3]:
import numpy as np
from tkinter import Tk
import pandas as pd
#Bug?:
#wenn der Fehlerwert in einem fehler array zb: err=[[1,2],[0.0,0.1]] 0 ist 
#wird überhaupt kein fehler angegebnen und nicht val\pm0.0

def table(align,head,val,err,cap,lab,nk,longtable=True,copy=False):
    #align: entweder 'l','c','r'
    #align="c"
    
    #head: np.array an titeln der spalten['a','b','c']
    #head=np.array(['a','b','c'])
    
    #val: np.array mit [] mit den Values zu den titeln[[1,2,3],[2,4,6],[3,6,9]]
    #val=np.array([[1,3,3],[9,4,6],[3,3,4]])
    
    #err: np.array mit [] zu den Values,[0] wenn keine error existieren
    #,wenn das array 1 wert besitz wird dieser für alle genommen ([[0.1,0.2,0.1],[0],[0.1]],dtype=object)
    #err=np.array([[0.1,0.2,0.1],[0],[0.1]],dtype=object)

    #cap: die caption
    #cap="jo"
    
    #lab: label mit tab: davor
    #lab="1a" => label: tab:1a
    
    #nk: np.array welches die Anzahl der Nachkommastellen beinhaltet für jeden Wert/Fehler 
    #oder skalar welches für alle werte die selbe Anzahl benutzt
    #nk=np.array([1,0,2]) oder nk=np.array([2])
    
    #longtable: Boolean welcher bestimmt ob eine longtable oder normale table erstellt wird
    #longtable=False
    
    
    #------------------------------------------
    
    
    #Datentypen ggf auf np.array ändern
    
    if type(head)!=np.ndarray:
        head=np.array(head)
    
    if type(val)!=np.ndarray:
        val=np.array(val)
    #Datentypen der Unterdaten ggf auf np.array ändern
    for i,array in enumerate(val): 
        if type(array)!=np.ndarray:
            val[i]=np.array(array)
    
    #Datentypen der Unterdaten ggf auf np.array ändern
    if type(err)!=np.ndarray:
        err=np.array(err,dtype=object)

    
    
    for i,array in enumerate(err): 
        if (type(array)==int or type(array)==float):
            err[i]=np.array([array])
        if type(array)==list:
            err[i]=np.array(array)
    
    if (type(nk)==int or type(nk)==float):
        nk=[nk]
    if type(nk)!=np.ndarray:
        nk=np.array(nk)

    #------------------------------------------
    
    
    #Überprüfen der Dims von Head
    if not len(head)==len(val):
        print("Head:")
        print(head)
        print("Länge von Val:")
        print(len(val))
        sys.exit("Die Länge von head stimmt nicht mit der Länge von Val überein")
    
    #Überprüfen der Dims von Values
    test=np.empty_like(val,dtype='int32')
    
    for i,array in enumerate(val): 
        test[i]=len(array)
            
    if not np.all(test == test[0]):
        print("Längen:")
        print(test)
        sys.exit("Die Längen der Arrays in Val stimmen nicht überein mit den Anderen")

    #Überprüfen der Dims von Error
    test=np.empty_like(err,dtype='int32')
    for i,array in enumerate(err): 
        if type(array)==np.float64:
            err[i]=np.array([array])
            test[i]=1
        else:
            test[i]=len(array)
            
    if not np.all(np.logical_or((test == np.max(test)),(test == 1))):
        print("Längen:")
        print(test)
        sys.exit("Die Längen der Arrays in err stimmen nicht überein mit den Anderen oder 1")
    
    
    #------------------------------------------
    
    
    #Erstellen des Arrays nk falls nk ein skalar ist
    if(len(nk)==1):
        nk=np.full(len(val),nk)
        
    
    #Erklären der funktion welche alles rundet
    def rundennachkomma(a,n):
    #a ist array welches gerundet werden soll
    #n ist die anzahl der nachkommastellen
    #Erzeugt array mit den selben dimensionen wie a nur als string array(max 20 char)
        ar=np.empty_like(a,dtype='<U20')
        for i,v in enumerate(a):
            if isinstance(v, str):
                ar[i]=v
            else:
                ar[i]="{:.{}f}".format(v,n)
        return ar
    
    
    #Erstellen eines leeren String Arrays damit es direkt in den Tabellestring eingesetzt werden kann
    valstr=np.empty_like(val,dtype='<U20')
    #Runden der Values    
    for i,array in enumerate(val): 
        valstr[i]=rundennachkomma(array,nk[i])
        
    #Runden der errors     
    for i,array in enumerate(err): 
        err[i]=rundennachkomma(array,nk[i])  
    
    
    #------------------------------------------
    
    #Erstellen eines leeren String Arrays damit es direkt in den Tabellestring eingesetzt werden kann
    fehler=np.zeros_like(valstr,dtype='<U20')
    
    #"drehen" des Arrays damit des richtig in die Tabelle eingesetzt werden kann und fehler bestimmt werden kann
    valstr=np.transpose(valstr)
    
    #Ersetzten von skalaren durch arrays in fehler
    for i,array in enumerate(err): 
        if(len(array)==len(valstr)):
            fehler[i]=array
        elif(len(array)==1):
            fehler[i]=np.full(len(valstr),array)
        else:
            print("err array:")
            print(array)
            print("val länge:")
            print(len(valstr))
            sys.exit("Die Länge des err arrays entsprict nicht der Länge des val arrays")

    
    #"drehen" des Arrays damit es richtig in die Tabelle eingesetzt werden kann
    fehler=np.transpose(fehler)
    
    #------------------------------------------
    
    
    if longtable==True:
        #Anfangen des tex Strings und Setup der Longtable
        a=(align+"|")*(len(val)-1)+align
        tex="\\begin{longtable}{"+a+"} \n \n\\caption{"+cap+"}\\\ \n \n"

        for string in head:
            tex+=string+"\t &"

        tex=tex[:-1]+"\\\\"+"\\hline\\hline"+"\n"+"\\endfirsthead \n \n\\caption{Fortsetzung "+cap+"}\\\ \n \n"

        for string in head:
            tex+=string+"\t &"
        tex=tex[:-1]+"\\\\"+"\\hline\hline\n\endhead"+"\n"+"\hline\n\multicolumn{"+str(len(val))+"}{r}{Weiter auf der n{\"a}chsten Seite}\\\n\endfoot\hline\n\multicolumn{"+str(len(val))+"}{r}{Ende} \\\n\endlastfoot\n \n"
    else:
        a=(align+"|")*(len(val)-1)+align
        tex="\\begin{table}[H] \n\\centering \n\\caption{"+cap+"} \n\\begin{tabular}{"+a+"} \n"
        
        for string in head:
            tex+=string+"\t &"
        tex=tex[:-1]+"\\\\"+"\\hline"+"\n"
    
    
    #------------------------------------------
    
    
    #Einsetzen von values und fehler in die Tabelle als gerundete Strings
    #itteration durch die values arrays
    for i,array in enumerate(valstr):
        #itteration durch die values der values arrays
        for j,v in enumerate(array):
            #Schreiben des values ohne den Fehler
            if(float(fehler[i,j])==0):
                tex=tex+v+"\t&"
            #Schreiben des values mit dem Fehler
            else:
                tex=tex+v+"$\\pm$"+fehler[i,j]+"\t&"
            
        tex=tex[:-1]+"\\\\ \n"
    
    #entfernen der letzten Chars damit keine leere Zeile erstellt wird durch den letzten Zeilenumbruch
    tex=tex[:-4]+"\n"
    
    
    #------------------------------------------
    
    
    if longtable==True:
        #beenden der Longtable
        if lab=="":
            tex=tex+"\n\\end{longtable}"
        else:
            tex=tex+"\\label{tab:"+lab+"} \n \n\\end{longtable}"
    else:
        if lab=="":
            tex=tex+"\\end{tabular} \n\\end{table}"
        else:
            tex=tex+"\\label{Tab:"+lab+"} \n\\end{tabular} \n\\end{table}"
    
    
    #------------------------------------------
    if copy:
        df = pd.DataFrame([tex])
        df.to_clipboard(excel=True, sep=None,index=False,header=False)
        #r = Tk()
        #r.withdraw()
        #r.clipboard_clear()
        #r.clipboard_append(tex)
        #r.update() # now it stays on the clipboard after the window is closed
        #r.destroy()
    
    return tex

In [8]:
def bsp1_code():
    align='c'
    head=["val1","val2","val3"]
    val=[[10,10,10],[10,10,10],[20,10.1,10.2]]
    err=[1,2,[1,2,3]] 
    cap="Caption"
    lab="Label"
    nk=2
    print(table(align,head,val,err,cap,lab,nk,longtable=False,copy=True))
def bsp1():
    print("align='c'")
    print("head=[\"val1\",\"val2\",\"val3\"]")
    print("val=[[10,10,10],[10,10,10],[20,10.1,10.2]]")
    print("err=[1,2,[1,2,3]]")
    print("cap=\"Caption\"")
    print("lab=\"Label\"")
    print("nk=2")
    print("print(table(align,head,val,err,cap,lab,nk,longtable=False,copy=False))")
#bsp1_code()

\begin{table}[H] 
\centering 
\caption{Caption} 
\begin{tabular}{c|c|c} 
val1	 &val2	 &val3	 \\\hline
10.00$\pm$1.00	&10.00$\pm$2.00	&20.00$\pm$1.00	\\ 
10.00$\pm$1.00	&10.00$\pm$2.00	&10.10$\pm$2.00	\\ 
10.00$\pm$1.00	&10.00$\pm$2.00	&10.20$\pm$3.00	
\label{Tab:Label} 
\end{tabular} 
\end{table}


In [5]:
def bsp2_code():
    v1=np.linspace(1,10,5)
    v2=np.linspace(9.9,10,5)
    v1_err=np.linspace(1,2,5)
    v2_err=np.linspace(0.001,0.01,5)
    align='c'
    head=["val1","val2"]
    val=[v1,v2]
    err=[v1_err,v2_err] 
    cap="Caption"
    lab="Label"
    nk=[1,4]
    print(table(align,head,val,err,cap,lab,nk,longtable=True,copy=False))
def bsp2():
    print("v1=np.linspace(1,10,5)")
    print("v2=np.linspace(9.9,10,5)")
    print("v1_err=np.linspace(1,2,5)")
    print("v2_err=np.linspace(0.001,0.01,5)")
    print("align='c'")
    print("head=[\"val1\",\"val2\"]")
    print("val=[v1,v2]")
    print("err=[v1_err,v2_err]")
    print("cap=\"Caption\"")
    print("lab=\"Label\"")
    print("nk=[1,4]")
    print("print(table(align,head,val,err,cap,lab,nk,longtable=True,copy=False))")
#bsp2_code()

\begin{longtable}{c|c} 
 
\caption{Caption}\\ 
 
val1	 &val2	 \\\hline\hline
\endfirsthead 
 
\caption{Fortsetzung Caption}\\ 
 
val1	 &val2	 \\\hline\hline
\endhead
\hline
\multicolumn{2}{r}{Weiter auf der n{"a}chsten Seite}\
\endfoot\hline
\multicolumn{2}{r}{Ende} \
\endlastfoot
 
1.0$\pm$1.0	&9.9000$\pm$0.0010	\\ 
3.2$\pm$1.2	&9.9250$\pm$0.0033	\\ 
5.5$\pm$1.5	&9.9500$\pm$0.0055	\\ 
7.8$\pm$1.8	&9.9750$\pm$0.0078	\\ 
10.0$\pm$2.0	&10.0000$\pm$0.0100	
\label{tab:Label} 
 
\end{longtable}


In [6]:
def bsp3_code():
    v1=["str1","str2","str3"]
    v2=[1,2,3]
    align='c'
    head=["","Werte"]
    val=[v1,v2]
    err=[0,np.array([0.5])] 
    cap="Caption"
    lab="Label"
    nk=[1337,4]
    print(table(align,head,val,err,cap,lab,nk,longtable=False,copy=False))
def bsp3():
    print("v1=[\"str1\",\"str2\",\"str3\"]")
    print("v2=[1,2,3]")
    print("align='c'")
    print("head=["",\"Werte\"]")
    print("val=[v1,v2]")
    print("err=[0,0.5]")
    print("cap=\"Caption\"")
    print("lab=\"Label\"")
    print("nk=[1337,4]")
    print("print(table(align,head,val,err,cap,lab,nk,longtable=False,copy=False))")
#bsp3_code()

In [2]:
def minipage(tabellen,captions):
    anzahl=len(tabellen)
    tabulare=[t.split("\n",3)[-1][:-13] for t in tabellen]
    string=""
    for tabelle,caption in zip(tabulare,captions):
        string+=r"\begin{minipage}[t]{"+str(1/(anzahl+0.1))+r"\textwidth}"+"\n"
        string+="\t"+r"\captionsetup{type=figure}"+"\n"
        string+="\t"+r"\centering"+"\n"
        string+="\t\t"+r"\captionof{table}{"+caption+r"}"+"\n"
        string+=tabelle+"\n"
        string+=r"\end{minipage}"+"\n"
        string+=r"\quad"+"\n"
    string=string[:-6]
    return string